In [27]:
import pandas as pd
import numpy as np
import datetime as dt
import string

In [43]:
def get_clean_brands(brands_df):
    '''
    This function takes in a dataframe with brands and remaps all brands to clean brands.
    For example, if the data contains brands of "xiao mi" and "xiaomi", all brands will be renamed to "xiao mi"
    :param brands_df:
    :return brands_df: dataframe of clean brands
    '''
    before_brands = brands_df['brand'].unique()
    # if number of brands == 0, just return brands_df
    if before_brands.shape[0] == 0:
        return brands_df
    # vectorized ambda function to replace whitespace inside nparray
    replace = lambda x: x.replace(" ", "")
    vfunc = np.vectorize(replace)
    print(before_brands)
    brands_removed_whitespace = vfunc(before_brands)
    count_of_brands = Counter(brands_removed_whitespace)
    # get brands who have count > 1 (number of version for brands more than 1)
    brands_more_than_1 = []
    for brand, counter in count_of_brands.items():
        if counter > 1:
            brands_more_than_1.append(brand)
    # make dictionary to remap brand names
    # for e.g "xiaomi" and "xiao mi" will now all be renamed to "xiao mi"
    before_brands = list(before_brands)
    brands_remap = {}
    for brand in before_brands:
        strip_space_brand = string.replace(brand, " ", "")
        if brand != strip_space_brand and strip_space_brand in brands_removed_whitespace and strip_space_brand in brands_more_than_1:
            brands_remap[strip_space_brand] = brand
    brands_df['brand'] = brands_df['brand'].apply(
        lambda brand: brands_remap[brand] if brand in brands_remap.keys() else brand)
    return brands_df

In [82]:
country_list = ['ID']
for country in country_list:
    df = pd.read_csv('{}_data.csv'.format(country))
    df['grass_date'] = pd.to_datetime(df['grass_date'])
    df_info = df[df['grass_date'] == dt.datetime(2019, 9, 30)]

    df_inv_count = df_info.groupby('supplier_name')['stock_on_hand'].sum().reset_index()
    df_inv_count.columns = ['supplier_name', 'inv_count']
    
    df_inv_sorting = df_info.groupby(['category_cluster', 'supplier_name'])['inventory_value_usd'].sum().reset_index()
    df_inv_sorting0 = df_inv_sorting.sort_values(['supplier_name', 'inventory_value_usd'], ascending=False)
    df_inv_sorting1 = df_inv_sorting0.groupby('supplier_name')['supplier_name', 'category_cluster'].head(1)

    df_inv_sum = df_info.groupby('supplier_name')['inventory_value_usd',].sum().reset_index()

    df_sku_count = df_info.groupby('supplier_name')['sku_id'].count().reset_index()
    df_sku_count.columns = ['supplier_name', 'no_skus_WH']
    df_info0 = df_sku_count.merge(df_inv_count, on=['supplier_name'], how='left')
    
    
    
    
    
    df_payment = df_info.groupby('supplier_name')['supplier_name', 'payment_terms'].head(1).reset_index(drop=True)
    df_sep = df[(df['grass_date'] >= dt.datetime(2019, 9, 1)) & (df['grass_date'] <= dt.datetime(2019, 9, 30))]
    df_sep.drop_duplicates(inplace=True)
    brands_df = df_sep[['supplier_name', 'brand']]
    brands_df = brands_df[brands_df.brand != 'nan']
    number_of_top_brands = 3
    supplier_names = set(brands_df['supplier_name'].to_list())
    brand_count_df = brands_df.groupby(['supplier_name', 'brand']).size()
    # get nlargest per subgroup
    brand_count_top_3_df = brand_count_df.groupby(['supplier_name'], group_keys=False).apply(lambda subgroup: subgroup.nlargest(3))
    # drop count column and reset_index
    brand_count_top_3_df = brand_count_top_3_df.reset_index().drop(columns=0)
    # final brands dataframe in desired output
    top_brands_cols = ['supplier_name', 'brand_1', 'brand_2', 'brand_3']
    top_brands_df = pd.DataFrame(columns=top_brands_cols)
    for supplier in supplier_names:
        slice_df = brand_count_top_3_df[(brand_count_top_3_df.supplier_name == supplier)].reset_index()
        three_brands = dict()
        for i in range(number_of_top_brands):
            key = 'brand_{}'.format(i + 1)
            try:
                brand = slice_df.iloc[i].brand
                three_brands[key] = brand
            except IndexError:
                three_brands[key] = 'n.a.'
        row_list = [supplier, three_brands['brand_1'], three_brands['brand_2'],
                    three_brands['brand_3']]
        row_to_append = pd.DataFrame([row_list], columns=top_brands_cols)
        top_brands_df = top_brands_df.append(row_to_append)

    top_brands_df = top_brands_df.reset_index(drop=True)
    top_brands_df.fillna('n.a.', inplace=True)
    df_info1 = df_info0.merge(df_inv_sum, on=['supplier_name'], how='left')
    df_info2 = df_info1.merge(top_brands_df, on=['supplier_name'], how='left')
    df_info3 = df_info2.merge(df_payment, on=['supplier_name'], how='left')
    df_total_sum = df.groupby(['supplier_name', 'grass_date']).sum().reset_index()
    df_total_sum['grass_date'] = df_total_sum['grass_date'].dt.strftime('%Y-%m-%d')
    df_cogs = pd.pivot_table(df_total_sum, values='cogs_usd', index='supplier_name', columns='grass_date').reset_index()

    df_payables = pd.pivot_table(df_total_sum, values='acct_payables_usd', index='supplier_name', columns='grass_date').reset_index()

    df_inv_value = pd.pivot_table(df_total_sum, values='inventory_value_usd', index='supplier_name', columns='grass_date').reset_index()

    df_inbound = pd.pivot_table(df_total_sum, values='inbound_value_usd', index='supplier_name', columns='grass_date').reset_index()
    df_info3.to_csv('{}_tracking_tab_v2.csv'.format(country), encoding="utf-8-sig")
    df_payables.to_csv('{}_payables_v2.csv'.format(country), encoding="utf-8-sig")
    df_cogs.to_csv('{}_cogs_v2.csv'.format(country), encoding="utf-8-sig")
    df_inv_value.to_csv('{}_inventory_value_v2.csv'.format(country), encoding="utf-8-sig")
    df_inbound.to_csv('{}_inbound_v2.csv'.format(country), encoding="utf-8-sig")



c:\users\yongshin.park\envs\replenishment\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df_info3